<a href="https://colab.research.google.com/github/scw0310/SEED/blob/main/ADF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
!pip install pandas statsmodels
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
!pip install arch
from statsmodels.tsa.stattools import adfuller, kpss
from arch.unitroot import PhillipsPerron

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 종속변수

In [83]:
df = pd.read_csv('/content/drive/MyDrive/EWHA/SEED/2024-2/SEED_24_2_논문2조/insurance.csv').dropna()
df.columns.values[0] = 'date'
insurance_col = ['date', '변액종신', '변액연금', '변액유니버셜']
insurance = df[insurance_col]

insurance.head()

,date,변액종신,변액연금,변액유니버셜
1,2018_11,0.00850,0.01002,0.00987
2,2018_12,0.00846,0.01009,0.00993
3,2019_01,0.01570,0.01061,0.01151
4,2019_02,0.01583,0.00976,0.01062
5,2019_03,0.01229,0.01074,0.01272


## 기본통계량

In [84]:
insurance.describe()

,변액종신,변액연금,변액유니버셜
count,50.000000,50.000000,50.000000
mean,0.008498,0.010531,0.008625
std,0.004817,0.002464,0.002467
min,0.004200,0.006490,0.004050
25%,0.005565,0.008700,0.006428
50%,0.006835,0.010125,0.009205
75%,0.008490,0.011845,0.010597
max,0.026990,0.019270,0.012720


## 차분, 추가 차분

In [85]:
from statsmodels.tsa.stattools import adfuller

insurance['변액유니버셜_diff'] = insurance['변액유니버셜'].diff()
insurance['변액유니버셜_diff2'] = insurance['변액유니버셜_diff'].diff()

insurance['변액종신_diff'] = insurance['변액종신'].diff()

<ipython-input-85-2e7abc72f220>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insurance['변액유니버셜_diff'] = insurance['변액유니버셜'].diff()
<ipython-input-85-2e7abc72f220>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insurance['변액유니버셜_diff2'] = insurance['변액유니버셜_diff'].diff()
<ipython-input-85-2e7abc72f220>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [86]:
insurance.head()

,date,변액종신,변액연금,변액유니버셜,변액유니버셜_diff,변액유니버셜_diff2,변액종신_diff
1,2018_11,0.00850,0.01002,0.00987,NaN,NaN,NaN
2,2018_12,0.00846,0.01009,0.00993,0.00006,NaN,-0.00004
3,2019_01,0.01570,0.01061,0.01151,0.00158,0.00152,0.00724
4,2019_02,0.01583,0.00976,0.01062,-0.00089,-0.00247,0.00013
5,2019_03,0.01229,0.01074,0.01272,0.00210,0.00299,-0.00354


## 단위근 검정 : ADF, PP, KSPP

In [87]:
def adf_test(data):
    adf_result = adfuller(data.dropna())
    print(f'ADF Statistic: {adf_result[0]}')
    print(f'p-value: {adf_result[1]}')
    print('Critical Values:')
    for key, value in adf_result[4].items():
        print(f'   {key}: {value}')

def pp_test(data):
    pp_result = PhillipsPerron(data.dropna())
    print('\n PP Test Results:')
    print(f'   Test Statistic: {pp_result.stat}')
    print(f'   p-value: {pp_result.pvalue}')
    print(f'   Lags Used: {pp_result.lags}')

def kpss_test(data, regression='c'):
    kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)
    print('\n KPSS Test Results:')
    print(f'   KPSS Statistic: {kpss_stat}')
    print(f'   p-value: {p_value}')
    print(f'   Lags Used: {lags}')
    print('   Critical Values:')
    for key, value in critical_values.items():
        print(f'      {key}: {value}')

In [88]:
#변액종신
data = insurance['변액종신']

adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -3.7290688175229594
p-value: 0.003722365326632832
Critical Values:
   1%: -3.5714715250448363
   5%: -2.922629480573571
   10%: -2.5993358475635153

 PP Test Results:
   Test Statistic: -4.525741633891015
   p-value: 0.00017661905293200506
   Lags Used: 11

 KPSS Test Results:
   KPSS Statistic: 0.8390781062067301
   p-value: 0.01
   Lags Used: 4
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-87-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


ADF, PP : 정상 시계열

KSPP : 비정상 시계열. 추세 포함되었을 가능성 있음

In [89]:
data = insurance['변액종신_diff']

adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -6.538949029383517
p-value: 9.452564235578092e-09
Critical Values:
   1%: -3.5812576580093696
   5%: -2.9267849124681518
   10%: -2.6015409829867675

 PP Test Results:
   Test Statistic: -16.642158498946394
   p-value: 1.6311130518953583e-29
   Lags Used: 11

 KPSS Test Results:
   KPSS Statistic: 0.3009961681880071
   p-value: 0.1
   Lags Used: 32
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-87-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [90]:
#변액연금
data = insurance['변액연금']
adf_test(data)

ADF Statistic: -3.169969049829759
p-value: 0.021790134909846215
Critical Values:
   1%: -3.5714715250448363
   5%: -2.922629480573571
   10%: -2.5993358475635153


In [91]:
#변액유니버셜
data = insurance['변액유니버셜']
adf_test(data)

ADF Statistic: -1.0191343191300035
p-value: 0.7462216841059601
Critical Values:
   1%: -3.60098336718852
   5%: -2.9351348158036012
   10%: -2.6059629803688282


In [92]:
data = insurance['변액유니버셜_diff']
adf_test(data)

ADF Statistic: -2.5308135228055133
p-value: 0.10815591472073038
Critical Values:
   1%: -3.60098336718852
   5%: -2.9351348158036012
   10%: -2.6059629803688282


In [93]:
data = insurance['변액유니버셜_diff2']
adf_test(data)

ADF Statistic: -7.5013035252360405
p-value: 4.246501975830064e-11
Critical Values:
   1%: -3.596635636000432
   5%: -2.933297331821618
   10%: -2.6049909750566895


**굵은 텍스트**

변액종신 -> '변액종신_diff'

변액연금 -> 그대로

변액유니버셜 -> '변액유니버셜_diff2'

In [94]:
insurance[['date', '변액종신_diff', '변액연금', '변액유니버셜_diff2']].to_csv('종속변수.csv', index=False, encoding='utf-8-sig')

from google.colab import files
files.download('종속변수.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 독립변수

In [95]:
df = pd.read_csv('/content/drive/MyDrive/EWHA/SEED/2024-2/SEED_24_2_논문2조/독립변수.csv').drop(columns=['Unnamed: 6'])

In [96]:
# 공백이나 특수문자 제거
df.columns = df.columns.str.strip()  # 양쪽 공백 제거
df.columns = df.columns.str.replace('(', '')  # 괄호 제거
df.columns = df.columns.str.replace(')', '')  # 괄호 제거

# 다시 열 이름 확인
print(df.columns)

Index(['date', 'CD91일', '실업률', '동행지수순환변동치', '총지수', 'KOSPI_증가'], dtype='object')


In [97]:
df.describe()

,CD91일,실업률,동행지수순환변동치,총지수,KOSPI_증가
count,51.000000,51.000000,51.000000,51.000000,51.000000
mean,1.502745,3.568627,99.447059,102.245216,2450.248431
std,0.826692,0.518263,1.291875,3.393543,424.456606
min,0.630000,2.600000,96.400000,98.884000,1754.640000
25%,0.780000,3.150000,98.800000,99.641000,2102.595000
50%,1.470000,3.600000,99.600000,100.330000,2293.610000
75%,1.820000,3.900000,100.250000,103.955000,2798.330000
max,4.020000,4.800000,101.400000,109.260000,3296.680000


로그차분

In [98]:
import pandas as pd
import numpy as np

# 로그 차분을 적용할 칼럼들
columns_to_diff = ['CD91일', '실업률', '동행지수순환변동치', '총지수', 'KOSPI_증가']

# 각 칼럼에 로그 차분 적용
for col in columns_to_diff:
    df[col] = (df[col]).diff()

# 데이터 확인
df.head(10)

,date,CD91일,실업률,동행지수순환변동치,총지수,KOSPI_증가
0,2018/10,NaN,NaN,NaN,NaN,NaN
1,2018/11,0.03,-0.2,-0.2,-0.711,67.17
2,2018/12,0.20,-0.2,-0.1,-0.342,-55.82
3,2019/01,-0.04,0.3,0.1,-0.104,163.81
4,2019/02,0.00,0.0,-0.5,0.427,-9.41
5,2019/03,0.03,-0.1,-0.1,-0.190,-54.77
6,2019/04,-0.04,0.3,-0.2,0.360,62.92
7,2019/05,-0.02,-0.2,0.3,0.171,-161.85
8,2019/06,-0.04,0.0,0.0,-0.161,88.88
9,2019/07,-0.12,0.0,0.1,-0.304,-106.07


CD91일 계절성 차분 실행

In [99]:
data = df['CD91일']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -1.4406101517592147
p-value: 0.562659802818095
Critical Values:
   1%: -3.5812576580093696
   5%: -2.9267849124681518
   10%: -2.6015409829867675

 PP Test Results:
   Test Statistic: -3.551460891416731
   p-value: 0.0067597452480446804
   Lags Used: 11

 KPSS Test Results:
   KPSS Statistic: 0.6635359037155456
   p-value: 0.01686037238949585
   Lags Used: 4
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


In [100]:
# 2차 로그 차분 수행
df['CD91일_diff2'] = (df['CD91일']).diff()

df.head()

,date,CD91일,실업률,동행지수순환변동치,총지수,KOSPI_증가,CD91일_diff2
0,2018/10,NaN,NaN,NaN,NaN,NaN,NaN
1,2018/11,0.03,-0.2,-0.2,-0.711,67.17,NaN
2,2018/12,0.20,-0.2,-0.1,-0.342,-55.82,0.17
3,2019/01,-0.04,0.3,0.1,-0.104,163.81,-0.24
4,2019/02,0.00,0.0,-0.5,0.427,-9.41,0.04


In [101]:
data = df['CD91일_diff2']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -3.709830336895313
p-value: 0.003976987271888764
Critical Values:
   1%: -3.5812576580093696
   5%: -2.9267849124681518
   10%: -2.6015409829867675

 PP Test Results:
   Test Statistic: -12.833957546728003
   p-value: 5.793796947356008e-24
   Lags Used: 11

 KPSS Test Results:
   KPSS Statistic: 0.18776634376298865
   p-value: 0.1
   Lags Used: 19
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-87-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [102]:
data = df['실업률']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -8.517629762471717
p-value: 1.1228393747924649e-13
Critical Values:
   1%: -3.5714715250448363
   5%: -2.922629480573571
   10%: -2.5993358475635153

 PP Test Results:
   Test Statistic: -9.105992461684659
   p-value: 3.5015389954497015e-15
   Lags Used: 11

 KPSS Test Results:
   KPSS Statistic: 0.08335481099656361
   p-value: 0.1
   Lags Used: 4
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-87-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [103]:
data = df['동행지수순환변동치']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -4.1942283820627235
p-value: 0.0006738806150507593
Critical Values:
   1%: -3.5778480370438146
   5%: -2.925338105429433
   10%: -2.6007735310095064

 PP Test Results:
   Test Statistic: -2.9814566047110054
   p-value: 0.0366627841003092
   Lags Used: 11

 KPSS Test Results:
   KPSS Statistic: 0.12472443272062543
   p-value: 0.1
   Lags Used: 3
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-87-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [104]:
data = df['총지수']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -5.04950585403135
p-value: 1.769497256706342e-05
Critical Values:
   1%: -3.5714715250448363
   5%: -2.922629480573571
   10%: -2.5993358475635153

 PP Test Results:
   Test Statistic: -5.196000346682396
   p-value: 8.958904523943184e-06
   Lags Used: 11

 KPSS Test Results:
   KPSS Statistic: 0.7908547379679053
   p-value: 0.01
   Lags Used: 2
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-87-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [105]:
# 2차 로그 차분 수행
df['총지수_diff2'] = (df['총지수']).diff()

df.head()

,date,CD91일,실업률,동행지수순환변동치,총지수,KOSPI_증가,CD91일_diff2,총지수_diff2
0,2018/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018/11,0.03,-0.2,-0.2,-0.711,67.17,NaN,NaN
2,2018/12,0.20,-0.2,-0.1,-0.342,-55.82,0.17,0.369
3,2019/01,-0.04,0.3,0.1,-0.104,163.81,-0.24,0.238
4,2019/02,0.00,0.0,-0.5,0.427,-9.41,0.04,0.531


In [106]:
data = df['총지수_diff2']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -7.4222099305191325
p-value: 6.69037532432337e-11
Critical Values:
   1%: -3.5778480370438146
   5%: -2.925338105429433
   10%: -2.6007735310095064

 PP Test Results:
   Test Statistic: -14.46459019463013
   p-value: 6.759698906746915e-27
   Lags Used: 11

 KPSS Test Results:
   KPSS Statistic: 0.4455337311467667
   p-value: 0.057528564160876434
   Lags Used: 39
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


In [107]:
data = df['KOSPI_증가']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -7.081527987024231
p-value: 4.65291541280176e-10
Critical Values:
   1%: -3.5714715250448363
   5%: -2.922629480573571
   10%: -2.5993358475635153

 PP Test Results:
   Test Statistic: -7.322676426377333
   p-value: 1.1828898841602764e-10
   Lags Used: 11

 KPSS Test Results:
   KPSS Statistic: 0.26984869922002497
   p-value: 0.1
   Lags Used: 1
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-87-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


**정리**

CD91일 -> CD91일_diff2

실업률, 동행지수순환변동치, KOSPI_증가 -> 그대로

총지수 -> 총지수_diff2

In [108]:
df_filtered = df[~df['date'].isin(['2018/10', '2018/11', '2018/12'])]

df_filtered[['date', 'CD91일_diff2', '실업률', '동행지수순환변동치', 'KOSPI_증가', '총지수_diff2']].to_csv('독립변수.csv', index=False, encoding='utf-8-sig')

from google.colab import files
files.download('독립변수.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>